# Load libraries

In [8]:
import pandas as pd
import numpy  as np
import string 
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Data import

In [9]:
pd.set_option('max_colwidth', None) 
df = pd.read_csv("~/Downloads/banData/MeTooHate.csv", nrows = 10000)
df.head()

,status_id,text,created_at,favorite_count,retweet_count,location,followers_count,friends_count,statuses_count,category
0,1046207313588236290,"Entitled, obnoxious, defensive, lying weasel. This thing is to make decisions that may affect my life? He is a women hater.",2018-09-30T01:17:15Z,5,1,"McAllen, TX",2253,2303,23856,0
1,1046207328113086464,Thank you and for what you did for the women and survivors this week. \n,2018-09-30T01:17:19Z,5,2,"Tampa, FL",2559,4989,19889,0
2,1046207329589493760,"Knitting (s) &amp; getting ready for January 19, 2019.",2018-09-30T01:17:19Z,0,0,"St Cloud, MN",16,300,9,0
3,1046207341283168256,"Yep just like triffeling women weaponized their poon!! Wonder if Kamala Harris ever extorted Willy Brown after throwing the poon on him, oh yeh, that how she got her first job me too is a JOKE!",2018-09-30T01:17:22Z,1,0,flyover country,3573,3732,38361,1
4,1046207347016826880,"No, the President wants to end movement posing as the movement.",2018-09-30T01:17:23Z,0,0,World,294,312,7635,0


## Extract Hate data

- Is there a sub-category?
- Check first 60 tweets labeled as 1
    - Negative sentiment expressed: 6, 8, 9, 11, 12, 14, 17, 20, 22, 23, 24, 26, 27, 28,30**,31, 32, 33, 34, 35, 36, 38, 40, 41, 42, 45, 46, 47, 48, 53, 54, 55, 56, 57, 58, 59, 60
    - Hate expressed:the rest


## External abusive word list

In [10]:
abusive_words_df = pd.read_csv('../raw_data/abusive_words_list.txt', delimiter = "\t").iloc[:,0:2]
abusive_words_df.columns = ["word", "neg_polarity"]


In [11]:
test = abusive_words_df.head()
test.word

0     horrible_noun
1    disgusting_adj
2        moron_noun
3      bastard_noun
4       stupid_noun
Name: word, dtype: object

In [14]:
word2 = [str(x).split("_")[0] for x in abusive_words_df.word]
word2 = pd.Series(word2)
word2
abusive_words_df["word2"] = word2
abusive_words_df

word3 = abusive_words_df["word2"].unique()
len(word3)
#'../raw_data/abusive_words_list.txt'
df.to_csv('../project_BAN/data/abusive_words.txt',header="words", index=None, sep=' ', mode='a')


In [261]:
df[["category"]].value_counts

<bound method DataFrame.value_counts of       category
0            0
1            0
2            0
3            1
4            0
...        ...
9995         0
9996         0
9997         1
9998         0
9999         0

[10000 rows x 1 columns]>

In [4]:
non_hate_text = df[df["category"]==0]["text"]
non_hate_text.iloc[0]

'Entitled, obnoxious, defensive, lying weasel. This thing is to make decisions that may affect my life? He is a women hater.   '

In [5]:
hate_text = df[df["category"]==1]["text"]

In [6]:
hate_text.iloc[0]


'Yep just like triffeling women weaponized their poon!! Wonder if Kamala Harris ever extorted Willy Brown after throwing the poon on him, oh yeh, that how she got her first job me too is a JOKE! '

In [7]:
hate_text.iloc[1]

'Save it for Fox Fake News—watched by racists, bigots &amp; other women-hating sexists like yourself. GOP is the party of Putin &amp; the party of Hate—besides women, GOP hates all POC, immigrants, and all ppl not straight. You’re the party of the entitled lying whiny white men.'

In [8]:
hate_text.iloc[2]

"As a white male in this  era I'm just scared that if I decide to rape or sexually assault a woman, it can be used against me years later. How am I supposed to ask a girl out?!"

In [9]:
hate_text.iloc[3]

'  I heard that  received some Shush/Slush money to settle rape charges by her aide who said she just could not remember where or when. . .'

In [10]:
hate_text.iloc[4]

"   If she hasn't, it's because of people like you shaming her! How dare you tell her how she is or should be feeling! That is why I HATE  the  movement. Along with all of the other movements! You rape these people to further a political objective! You are despicable!"

In [11]:
hate_text.iloc[5]

' Oh poor girl. You should have stayed disney star all your life. Because now you are being eaten by the industry. And in few years when your popularity falls, you will come with some  bs like other forgoten women in hollywood.'

In [12]:
hate_text.iloc[6]

'When  was just gaining momentum, a female friend of mine who was pregnant with her daughter said “what’s the big deal?  I’ve been raped”.  My response was “ You know I disagree with you 💯 “. How do you respond?  I was shocked!'

In [13]:
hate_text.iloc[7]

"She sounds like one horny bitch! Why it is always men who get blamed and framed for rapes? Am not talking about REAL rapes here but those that are hoaxes like Ford's! Women can rape men too! Just look at the sick  broads! How many look like normal, healthy &amp; happy women?"

In [14]:
hate_text.iloc[8]

" Trust me...Not like I did. I'm 61 &amp; to this day I cringe at some of the shit I did. I never sexually assaulted anyone. But, I did things that were flat out wrong under . If I ran into any ladies from 40 years ago, I'd take a knee to apologize. Kavanaugh needs to do same."

In [15]:
hate_text.iloc[9]

'.  you say you believe all survivors. \n\nWhy have you not made a statement on Keith Ellison’s multiple abuse allegations and stood up for his abused ?  \n\nYou SAID you believe all survivors. This is your stateof Minnesota. You need to address this  \n\n'

In [16]:
hate_text.iloc[10]

'In all reality the current United States government is traumatizing the fuck out of a lot of American people. We have rapists in the White House. What more of a fucked up message do you want to send? ✌💛🤔🙏   '

In [17]:
hate_text.iloc[11]

"No, they are not afraid of the truth.  They don't give one rats ass about the truth.  They know he did it and they don't care.  \n \n\n\n\n\n"

In [18]:
hate_text.iloc[12]

'\n\n14 year old boy had a white woman accuse him of sexually assaulting her.  He was innocent but murdered anyway—because you know—.\n\nShe told the truth on her death bed over 60 years later.\n\n'

In [19]:
hate_text.iloc[13]

'\n\n14 year old boy had a white woman accuse him of sexually assaulting her.  He was innocent but murdered anyway—because you know—.\n\nShe told the truth on her death bed over 60 years later.\n\n'

In [20]:
hate_text.iloc[14]

" I suffer from it and fibromyalgia, due to Domestic Violence that occurred since '92.\n \n\n \n"

In [21]:
hate_text.iloc[15]

'\n\n14 year old boy had a white woman accuse him of sexually assaulting her.  He was innocent but murdered anyway—because you know—.\n\nShe told the truth on her death bed over 60 years later.\n\n'

In [22]:
hate_text.iloc[16]

'When women stop performing in a provacative way, then talk to me about the  movement!'

In [23]:
hate_text.iloc[17]

"Is god damn scary. \nVoicing your opinion on the subject, where you're not 100% on the  train will give you loads of scorn and hate. \nSimply because you have issues with certain parts is ludicrous to say the least. \nThe risk of losing everything 6/?"

In [24]:
hate_text.iloc[18]

"Are we really supposed to believe all women all of the time?\n\nCertainly that isn't truly what  is all about."

In [25]:
hate_text.iloc[19]

'This fucking pisses me off. How dare Cheeto.\n\n      '

In [26]:
hate_text.iloc[19]

'This fucking pisses me off. How dare Cheeto.\n\n      '

In [27]:
hate_text.iloc[20]

'N.J. politician, 83, enters  movement with sex-assault admission from 70 years ago when she was 13'

In [28]:
hate_text.iloc[21]

'at19 i had a bf that was 28.. He wanted kids i didnt so i started birth controll.. My friend got mad at me and told him.. He came home from work early that day and tried to stab me in my vagina with a railroad spike he had on the shelf.. I fought hard and got a black eye'

In [29]:
hate_text.iloc[22]

'I really wish I would hug Dr. Christine Blasey Ford. It breaks my heart to see her try so hard to maintain her composure. We stand behind you, girl. I look up to you so much.   '

In [30]:
hate_text.iloc[23]

'       It would cost 100s of thousands to have to mount a legal defense against these types of false allegations. And believe me, there are plenty of nutso parents out there that would see a way to use the  movement to cause harm. That’s its legacy.'

In [31]:
hate_text.iloc[24]

'Does the  work if a man is used/abused by a woman?'

In [32]:
hate_text.iloc[25]

'BELIEVE HER!!!!!!!!!!!!!!!!!!!!!!!!!!!!  !!!!!!!!!!!!!!!! BELIEVE BELIEVE BELIEVE!!!!!!!!  \n\nYou fucking hypocrite loser '

In [33]:
hate_text.iloc[26]

'People will stop believing victims of sexual abuse.  is a smear to take down powerful men, has nothing to do with actual  sex harassment or sex abuse.'

In [34]:
hate_text.iloc[27]

"Are you going to fire the porn professor. He's a psychology professor what the hell.He watch's women being abused were is the metoo people at. "

In [35]:
hate_text.iloc[28]

'Good. Then I expect you to stand side by side with Juanita Broaddrick so she may have the long overdue justice she deserves '

In [36]:
hate_text.iloc[29]

"1 assault on a woman is an assault against all women. \n\nYou don't get to silence us so you can stay comfortable. \nYou don't get to decide when we come forward. \nYou don't get to rewrite our stories. \nYou don't own the truth. \nYou weren't there. \n\nEnough is enough."

In [37]:
hate_text.iloc[30]

'I used to dream of marrying a nice man and living happily ever after, like the romance books.\nNow I dream of having a fantastic job, a lovely home and children. I don’t need a spouse to make me happy, I can learn how to do it on my own.\n\nThat’s what my own  events have done'

In [38]:
hate_text.iloc[31]

'She literally starred in a soft porn series where a woman was dominated and degraded by a powerful white man.\n\nHolllywood Hypocrite! '

In [39]:
hate_text.iloc[32]

"Ouch! Wtf! It's not something you want to have a competition in anyways....why would she even suggest that it is? I'm always here if you need someone hun. ❤"

In [40]:
hate_text.iloc[33]

'If women all stuck together we would be so powerful!    '

In [41]:
hate_text.iloc[34]

"Does that sound extreme? My point is that it isn't extreme at all."

In [42]:
hate_text.iloc[35]

'  is BULLSHIT'

In [43]:
hate_text.iloc[36]

'Beautifully written. Thank you to your wife for sharing her story. My heart is with her'

In [44]:
hate_text.iloc[37]

'The outcome has already been decided. This is all a sham.  is already confirmed and hell is coming in November.       '

In [45]:
hate_text.iloc[38]

"I can't help but wonder if Christine Blasey-Ford is experiencing a secondary trauma at having her name now forever associated with her attacker."

In [46]:
hate_text.iloc[39]

'I grew up watching her perform.\n\nNow I can stand beside her in her action.    '

In [47]:
hate_text.iloc[40]

'She could care less what you say about her. Call her a liar all you want. She listened to the wishes of Dr. Ford.'

In [48]:
hate_text.iloc[41]

" I believe you.\nI see you.\nI'm inspired by you.\nI am you.\n\n"

In [49]:
hate_text.iloc[42]

'This slipped under the radar - while the Kavanaugh hearings were taking place on Thursday, another  story was also being heard on Capitol Hill. A government employee spoke before a House committee about sexual harassment allegations.'

In [50]:
hate_text.iloc[43]

"In case you missed our most recent reader-submitted Op-Ed \nClick, 'This week' on our home page\n"

In [51]:
hate_text.iloc[44]

'“It’s time to give a damn, let’s work together!”  \n\n        …'

In [52]:
hate_text.iloc[45]

"Because  is true and powerful (thanks,  ❤️), I'm unearthing so much of my past. I just remembered that time a platonic male friend came to my house with a fish bat, repeatedly brandishing it in a threatening manner, and how calm I had to remain to get him out."

In [53]:
hate_text.iloc[46]

"   You're supporting Menenedez... the man that slept with underaged girls? Omg... "

In [54]:
hate_text.iloc[47]

'Proof that  and  only apply to alleged accusations against Republicans. What a bunch of fucking hypocrites.\nKavanaugh Protesters Refuse to Denounce Keith Ellison'

In [55]:
hate_text.iloc[48]

"  Yup, there's a huge untapped voter reserve of Old White Guys looking to minimize the threat of present or future  lawsuits + Stepford Wives indignant that they might be permitted to have some say about assaults on their bodies."

In [56]:
hate_text.iloc[49]

'IT’S AS IF  ISN’T REALLY ABOUT FAIRNESS AND EQUALITY AT ALL:\xa0“If your friend says she wants to cut off every dick in a five mile radius, let her!”\n\n'

In [57]:
hate_text.iloc[50]

'    What parents let 15 year olds out partying? Getting gang raped? Either it’s bullshit or let’s put SOME blame where it belongs. Tired of this  crap. That’s what we have police and courts for.'

In [58]:
hate_text.iloc[51]

"  lol. he's a thin-skinned mouse ain't he?!"

In [59]:
hate_text.iloc[52]

" is so blinded by partisanship she can't see just how desperately twisted she looks bringing this up every time CPC tank in polls. Scandal mongering on BS hasn't worked for three yr.Most Cdns aren't as stupid as she and CPC think and hate her hijacking"

In [60]:
hate_text.iloc[53]

'I Believe Her -I Stand With Her -'

In [61]:
hate_text.iloc[54]

'           (2) If that was what triggered her memory , then that is not reliable , her lawyers did her a great harm by not producing those therapy notes! On the other side those women who accosted Jeff Flake , they work for an activist group &amp; have cheapened the  movement !'

In [62]:
hate_text.iloc[55]

'One Thousand White Women by Jim Fergus - a long published gem I just discovered, resonates in light of  More on Books Can Save a Life '

In [63]:
hate_text.iloc[56]

'The  movement is going to make dudes sign waivers before they can start fucking 💀'

In [64]:
hate_text.iloc[57]

'I am a man; I once was just a boy. The father of one of my jr. high school friends tried to molest me (and others) with me one particular day that I am aware of. He plied us with jack daniels and quaaludes to the point we could barely walk while we believed his lies.'

In [65]:
hate_text.iloc[58]

'Sadly, many who are worked up about  are the same people who didn’t think twice about supporting  even though she threatened &amp; bullied women who have spoken out about her pervert husband  for years. '

In [66]:
hate_text.iloc[59]

'I get offered sex at work so much it’s disturbing. Girls are not innocent!'

In [67]:
hate_text.iloc[60]

"careful she's probably not ashamed to slap you with a  allegation if you get too close.  bring measuring tape 👌✌️"

In [199]:
# Labeling

# Sample data

In [ ]:
df_mini = df[df["category"]==1].iloc[0:61]
len(df_mini)

In [119]:
len(hate_line)

37

In [110]:
hate_line = [6, 8, 9, 11, 12, 14, 17, 20, 22, 23, 24, 26, 27, 28,30,31, 32, 33, 34, 35, 36, 38, 40, 41, 42, 45, 46, 47, 48, 53, 54, 55, 56, 57, 58, 59, 60]
label = [2 if i in hate_line else 1 for i in range(61)]
#label

In [115]:
df_mini["label"] = label
#df_mini.head(10)

In [252]:
tmp = df.iloc[df["ca"]][0:40]
tmp["label"] = 1
tmp

KeyError: 'ca'

# Re-labeling

# Litterature & github account

- label propagation:  https://towardsdatascience.com/semi-supervised-learning-how-to-assign-labels-with-label-propagation-algorithm-9f1683f4d0eb
- Lexicon of abusive words: https://github.com/uds-lsv/lexicon-of-abusive-words     
- Deep learning and clustering

# Alternative modeling plans

 - Manually relabel 2%-5% of tweets in category 1 (hate speech)
 - Apply label propagation algo to label the remaining tweets
 - All tweets labeled,  proceed with classification/DL model

# Data preprocessing



## Data cleaning

In [216]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /home/mimi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mimi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mimi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/mimi/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [233]:
text = hate_text.iloc[59]

# lowercase
text = text.lower() 

# remove numbers
text = ''.join(word for word in text if not word.isdigit())

# punctuation
#string.punctuation
for punctuation in string.punctuation:
    text = text.replace(punctuation, '') 
    
# stopwords
stop_words = set(stopwords.words('english'))


In [234]:
# tokenizing - transform to list
word_tokens = word_tokenize(text) 
text = [w for w in word_tokens if not w in stop_words] 
text

['get',
 'offered',
 'sex',
 'work',
 'much',
 '’',
 'disturbing',
 'girls',
 'innocent']

## Lemantizing

In [235]:
# Stemming or Lemmatizing- finding root word
# stemmer = PorterStemmer()
# stemmed = [stemmer.stem(word) for word in text]
lemmatizer = WordNetLemmatizer()
lemmatized = [lemmatizer.lemmatize(word) for word in text]
text = lemmatized
lemmatized


['get',
 'offered',
 'sex',
 'work',
 'much',
 '’',
 'disturbing',
 'girl',
 'innocent']

## Vectorizing

In [221]:
# text numerical ----> representation

## Bag of words: counting ocuurences of each word (mx with word in col)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)
X.toarray()   
#vectorizer.get_feature_names()  
#pd.DataFrame(X.toarray(),columns = vectorizer.get_feature_names())  

array([[0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0]])

In [223]:
## Tf-Idf: Term Frequency - Inverse Document Frequency --> importance of word in document--> frequency
# pros: - frequency robust to document length
#       - measures importance
# cons: doesn't capture context

# texts = ['i love football',
#          'football is a game i love',
#         'football football football']

tf_idf_vectorizer = TfidfVectorizer()
X = tf_idf_vectorizer.fit_transform(text)
#X.toarray()
#pd.DataFrame(X.toarray(),columns = tf_idf_vectorizer.get_feature_names())

In [ ]:
# Key paramters BOW and Td-Idf :
# max_df = exclude "corpus specific stopwords", most freq words
# min_df = exclude words that are very infrequent in the dataset
# max_features = specify the number of features to keep when vectorizing, useful to reduce the dimension of the data

In [225]:
### N-grams: Instead of considering individual words, 
#N-grams consists of considering word sequences. 
#This representation captures context. N is the number of words to be consiered as a one
texts =  ['i do not love football',
         'i love football not basketball']
tf_idf_vectorizer = TfidfVectorizer(ngram_range = (2,2))

X = tf_idf_vectorizer.fit_transform(texts)

X.toarray()

pd.DataFrame(X.toarray(),columns = tf_idf_vectorizer.get_feature_names())

/home/mimi/.pyenv/versions/3.8.12/envs/project_BAN/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,do not,football not,love football,not basketball,not love
0,0.631667,0.000000,0.449436,0.000000,0.631667
1,0.000000,0.631667,0.449436,0.631667,0.000000


# Test label propagation on minimal data

In [202]:
! pip install plotly

     |████████████████████████████████| 26.5 MB 5.5 MB/s            
  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)


In [224]:
# Visualization
import plotly.express as px # for data visualization
import plotly.graph_objects as go # for data visualization
import matplotlib.pyplot as plt # for displaying confusion matrix

# Skleran
from sklearn.metrics import classification_report # for model evaluation metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay # for showing confusion matrix
from sklearn.preprocessing import MinMaxScaler # for feature scaling
from sklearn.semi_supervised import LabelPropagation # for assigning labels to unlabeled data


# Useful notes

## Some features are:

- Vocabulary Richness
- Number of words per tweet
- punctuation/Character ratio
- emoji/Character ratio
- Contains abusive words
- Contains Words in Capital letters

In [229]:
def vocab_richness(text):
    tokens = word_tokenize(text)
    total_length = len(tokens)
    unique_words = set(tokens)
    unique_word_length = len(unique_words)
    return unique_word_length/total_length

vocab_richness = df_mini.text.apply(vocab_richness)

vocab_richness

3      0.902439
7      0.716667
49     0.883721
57     0.961538
60     0.793103
         ...   
447    1.000000
459    0.774194
462    0.875000
464    1.000000
466    0.954545
Name: text, Length: 61, dtype: float64

## Machine learning models
- Naive base fro classification
- LatentDirichletAllocation for clustering

## Deep learning model

https://kitt.lewagon.com/camps/773/lectures/content/06-DL_05-Natural-Language-Processing.html
    
X.shape = (n_sentences, max_sentence_length, embedding_dim)

steps:
    - tokenize text: tf.keras.preprocessing.text.Tokenizer
    - embedding layer (creating vector representation of each word): https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding
                - custome embedding : can be done but computationally intensive
                - independent embedding woth Word2vec (Gensim) --- transfer learning
                
Powerful embedding that is very fast and easy to train!

✅ you give it a list of sentences
✅ it automatically learns a representation - an embedding - for each word it was trained on!